<a href="https://colab.research.google.com/github/Novik-data-analyst/career-opportunity-cases/blob/main/Credit%20Scoring/Credit_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="alert alert-info"> <h4> Loan applicant reliability research </h4>

Bank credit department ordered new research. We need to determine, whether martial status and number of kids in household affects or credit payoff on time.

Analysis results would be included in credit scoring model - special system which evaluates borrower`s potential ability to pay-off loan on time.  <div/>



 <strong><em><h2>  Step 1. Loading file and studying information   </em></strong></h2>

In [ ]:
import pandas as pd
data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')
data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<div class="alert alert-info">
     <h4> Data description </h4>

   -children — number of kids on household
    days_employed — seniority in days
    dob_years — clients age in years
    education — clients highest achieved degree
    education_id — degree level id
    family_status — martial status
    family_status_id — martial status id
    gender — clients gender
    income_type — type of employment
    debt — does client had past-due payments before.
    total_income — monthly income
    purpose — loan purpose

  <div class="alert alert-info"> based on spreadsheet data, I found out that we have mismatch in columns 'days_unemployed' and 'total_income' - there`s less values that we have in other columns, which means missing data. <div/>

 <strong><em><h2>  Step 2. ETL </em></strong></h2>


<div class="alert alert-info"> Issues:
    
* children:
    
 1) changing value 20 (should be equal 2)
    
 2) changing value -1 on to 1
    
* days_employed:
    
 1) changing missing values on median value for the column
    
 2) there is big difference in numbers: people with income_type == senior greatly exceed rest ( minimal value is  300000+ whereas the rest are between -24 and up to -18000) .  We need to convert them from hours into days.
    
 3) we need to change rules on negative values - delete minus.
    
* dob_years:
    
 1) anomalus value dob_years = 0. We should change to average value in correlation to days employed.
    
* education:
    
 1) change every value to lower case to eliminate duplicates
    
* gender:
    
 1) change irrelevant XNA on M - just one case won`t affect sampling
    
* total_income:
    
 we got missed values , fill them according to age of customer and highest degreeе <div/>








<div class="alert alert-info"> Children: 1) Change inadequate values -1 on 1 and 20 on 2 <div/>

In [ ]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [ ]:
data.loc[data['children']== -1, 'children'] = 1
data.loc[data['children']== 20, 'children'] = 2


data['children'].value_counts()


0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


 <div class="alert alert-info"> Parts of values in column days_employed differs - i presume,
they shown in hours, but not days, as it meant in column - bring to equal time:  <div/>

In [ ]:
data.loc[data['days_employed'] > 300000, 'days_employed'] /= 24
data.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи



 <div class="alert alert-info"> Further, we change negative values - obviuosly, it is resault of incorrect input. <div/>

In [ ]:
data.loc[data['days_employed'] < 0, 'days_employed'] = -data['days_employed']
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи



 <div class="alert alert-info"> Next we fill missing values by median variable.  <div/>

In [ ]:
days_avg = data['days_employed'].median()
days_avg
data['days_employed'] = data['days_employed'].fillna(value = days_avg).astype('int')
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.479721,4393.983415,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,0.755528,5131.540470,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,0.000000,24.000000,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1025.000000,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,2194.000000,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,4779.000000,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,5.000000,18388.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


 <div class="alert alert-info"> Let`s research column of loan applicants age.  <div/>

In [ ]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

 <div class="alert alert-info"> Removing found anomal zero value   <div/>

In [ ]:
median_years = data['dob_years'].dropna().median()
data=data.replace({'dob_years': {0: median_years}})

In [ ]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

 <div class="alert alert-info"> bringing all value in column education to lower case     <div/>

In [ ]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

  <div class="alert alert-info"> Next in column "gender"  we change one fault value XNA   <div/>

In [ ]:
data.loc[data['gender']== 'XNA','gender'] = 'M'
data['gender'].value_counts()


F    14236
M     7289
Name: gender, dtype: int64

 <div class="alert alert-info"> Filling missing values in column total_income by median method based on income_type column.     <div/>

In [ ]:
print('Missing values before:', data['total_income'].isna().sum())

for income_type in data['income_type'].unique():
    median = data.loc[data['income_type'] == income_type, 'total_income'].median()
    print(income_type,median)
    data.loc[(data['total_income'].isna())&(data['income_type'] == income_type), 'total_income'] = median

print('Missing values after:', data['total_income'].isna().sum())

Missing values before: 2174
сотрудник 142594.39684740017
пенсионер 118514.48641164352
компаньон 172357.95096577113
госслужащий 150447.9352830068
безработный 131339.7516762103
предприниматель 499163.1449470857
студент 98201.62531401133
в декрете 53829.13072905995
Missing values after: 0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


## Changing object values on integer numbers

In [ ]:
df2 = data.copy()
df2['dob_years'] = data['dob_years'].astype('int')
df2['total_income'] = data['total_income'].astype('int')

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [ ]:
df2.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


# Removing duplicates

 <div class="alert alert-info"> I choose starting method for finding and deleting duplicates for all columns, because it`s hardly imaginable to find perfect match for all 12 columns in database. <div/>

In [ ]:
print('Amount of completely identical strings:',df2.duplicated().sum())

Amount of completely identical strings: 72


In [ ]:
df2 = df2.drop_duplicates()
print('Completely identical strings after duplicate delete:',df2.duplicated().sum())

Completely identical strings after duplicate delete: 0


 <div class="alert alert-info"> At ETL stage, we brought all degrees in column education to lower case, that way getting rid of all the duplicates   <div/>

In [ ]:
df2.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


# 'Loan purpose' column lemmatization

  <div class="alert alert-info"> At first, we define unique categories by which we would be lemmatizing. <div/>

In [ ]:
df2['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     767
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

 <div class="alert alert-info">
    I defined further categories :
    
1) consumer credit - here we including all purposes with words like wedding, repairs and etc.
    
2) education
    
3) car loan
    
4) commercial real estate - I define this part of real estate separately, due to different type of loan applicants - in commercial it is mostly self-employed enterpreneurs

5) realt estate mortgage. all real estate loans for personal purposes.<div/>

In [ ]:
from pymystem3 import Mystem
m = Mystem()
##Making list 'purpose.unique()'
purpose_list = df2['purpose'].unique()
print(purpose_list)
print()
print()
lemmas = []
##Each string in 'purpose_list' we lemmatize, getting lemms list and adding values to empty list'lemms', then we check
for i in purpose_list:
    lemma = m.lemmatize(i)
    lemmas.append(lemma)
#print(lemmas)
##Dividing categories by lemms: "Свадьба", "автомобиль", "образование", "коммерческая недвижимость", 'ремонт', 'недвижимость'
def purpose_change(purpose):
    lemmas_row = m.lemmatize(purpose)
    for i in lemmas_row:
        if 'автомоб' in i:
            return 'автомобиль'
        if 'недвижимость' in i:
            return  'недвижимость'
        if 'жил' in i:
            return  'недвижимость'
        if 'коммерч' in i:
            return  'коммерческая недвижимость'
        if 'свад' in i:
            return  'потребительский'
        if 'образов' in i:
            return 'образование'
        if 'ремонт' in i:
            return  'потребительский'

df2['purpose_def']= df2['purpose'].apply(purpose_change)
df2.head(10)

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']




,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_def
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,потребительский
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,потребительский
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


In [ ]:
lemmas

[['покупка', ' ', 'жилье', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['дополнительный', ' ', 'образование', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['образование', '\n'],
 ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'],
 ['покупка', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'],
 ['недвижимость', '\n'],
 ['строительство', ' ', 'недвижимость', '\n'],
 ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['жилье', '\n'],
 ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'],
 ['автомобиль'

Next, we`re checking that all lines categorized and no missed lines have left.

In [ ]:
df2['purpose_def'].isnull().sum()

0

# Categorization


  <div class="alert alert-info">
 <b> To begin with, we define categoties :</b>
    
1) education
    
2) family_status
    
4) income_type
    
5) purpose_def
    

 <b> For categories with numbers: </b>
    
1) days_employed
    
2) dob_years
    
3) total income
    
4) children
    <div/>


In [ ]:
education_dict=df2[['education','education_id']]
education_dict = education_dict.drop_duplicates().reset_index(drop = True)
education_dict.sort_values(by = 'education_id', ascending = True)

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [ ]:
family_dict=df2[['family_status','family_status_id']]
family_dict = family_dict.drop_duplicates().reset_index(drop = True)
family_dict.sort_values(by = 'family_status_id', ascending = True)

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [ ]:
income_dict = df2[['income_type']].drop_duplicates().reset_index(drop = True)
income_dict

,income_type
0,сотрудник
1,пенсионер
2,компаньон
3,госслужащий
4,безработный
5,предприниматель
6,студент
7,в декрете


In [ ]:
purpose_dict = df2[['purpose_def']].drop_duplicates().reset_index(drop = True)
purpose_dict

,purpose_def
0,недвижимость
1,автомобиль
2,образование
3,потребительский
4,коммерческая недвижимость



    

 <div class="alert alert-info">
    <b> 1) Days_employed categories:</b>
    
 1) от 24 до 730 дней - стажер
    
 2) 730 - 4797 - постоянный работник
    
 3) 4797 - ударник <div/>

In [ ]:
def days_employed_group(days):

    if days <= 730:
        return 'стажер'
    if days <= 4797:
        return 'постоянный работник'
    return 'ударник'
df2['days_employed_group'] = df2['days_employed'].apply(days_employed_group)
df2.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_def,days_employed_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,ударник
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,постоянный работник
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,ударник
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,постоянный работник
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,потребительский,ударник
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,постоянный работник
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,постоянный работник
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,стажер
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,потребительский,ударник
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,постоянный работник


In [ ]:
days_employed_dict = df2[['days_employed_group']].drop_duplicates().reset_index(drop = True)
days_employed_dict

,days_employed_group
0,ударник
1,постоянный работник
2,стажер



    
    
    

 <div class="alert alert-info">
    <b> 2) Возраст dob_years</b>
    

 1) до 22 - молодые
    
 2) от 22 до 60 - рабочий возраст
    
 3) от 60 пенсионный <div/>

In [ ]:
def dob_years_group(years):

    if years <= 22:
        return 'молодые'
    if years <= 60:
        return 'рабочий возраст'
    return 'пенсионный возраст'
df2['dob_years_group'] = df2['dob_years'].apply(dob_years_group)
df2.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_def,days_employed_group,dob_years_group
21515,1,467,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486,заняться образованием,образование,стажер,рабочий возраст
21516,0,914,42,высшее,0,женат / замужем,0,F,компаньон,0,322807,покупка своего жилья,недвижимость,постоянный работник,рабочий возраст
21517,0,404,42,высшее,0,гражданский брак,1,F,компаньон,0,178059,на покупку своего автомобиля,автомобиль,стажер,рабочий возраст
21518,0,15583,59,среднее,1,женат / замужем,0,F,пенсионер,0,153864,сделка с автомобилем,автомобиль,ударник,рабочий возраст
21519,1,2351,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949,покупка коммерческой недвижимости,коммерческая недвижимость,постоянный работник,рабочий возраст
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость,постоянный работник,рабочий возраст
21521,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль,ударник,пенсионный возраст
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость,постоянный работник,рабочий возраст
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль,постоянный работник,рабочий возраст
21524,2,1984,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047,на покупку автомобиля,автомобиль,постоянный работник,рабочий возраст


In [ ]:
dob_years_dict = df2[['dob_years_group']].drop_duplicates().reset_index(drop = True)
dob_years_dict

,dob_years_group
0,рабочий возраст
1,пенсионный возраст
2,молодые



    
    
    

 <div class="alert alert-info">
    <b> 3) Total income:</b>
    

 1) до 70000 - прожиточный минимум

 2) до 142594 - синий вортничок

 3) свыше - средний и высший класс
    
 <div/>


In [ ]:
def income_group(money):

    if money <= 70000:
        return 'прожиточный минимум'
    if money <= 142594:
        return 'синий воротничок'
    return 'средний и высший класс'
df2['income_group'] = df2['total_income'].apply(income_group)
df2.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_def,days_employed_group,dob_years_group,income_group
21515,1,467,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486,заняться образованием,образование,стажер,рабочий возраст,синий воротничок
21516,0,914,42,высшее,0,женат / замужем,0,F,компаньон,0,322807,покупка своего жилья,недвижимость,постоянный работник,рабочий возраст,средний и высший класс
21517,0,404,42,высшее,0,гражданский брак,1,F,компаньон,0,178059,на покупку своего автомобиля,автомобиль,стажер,рабочий возраст,средний и высший класс
21518,0,15583,59,среднее,1,женат / замужем,0,F,пенсионер,0,153864,сделка с автомобилем,автомобиль,ударник,рабочий возраст,средний и высший класс
21519,1,2351,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949,покупка коммерческой недвижимости,коммерческая недвижимость,постоянный работник,рабочий возраст,синий воротничок
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость,постоянный работник,рабочий возраст,средний и высший класс
21521,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль,ударник,пенсионный возраст,средний и высший класс
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость,постоянный работник,рабочий возраст,синий воротничок
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль,постоянный работник,рабочий возраст,средний и высший класс
21524,2,1984,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047,на покупку автомобиля,автомобиль,постоянный работник,рабочий возраст,синий воротничок


In [ ]:
income_group_dict = df2[['income_group']].drop_duplicates().reset_index(drop = True)
income_group_dict

,income_group
0,средний и высший класс
1,синий воротничок
2,прожиточный минимум



    
    
    
    

 <div class="alert alert-info">
    <b> 4) Children:</b>
    

 1) 0 - без детей

 2) 1 - один ребенок

 3) от 2 до 3 - несколько детей

 4) больше 4 - многодетные
    
    
 <div/>


In [ ]:
df2['children'].value_counts()

0    14090
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

In [ ]:
def child_status(child):

    if child == 0:
        return 'без детей'
    if child <= 2:
        return 'один-два ребенка'
    return 'многодетные'
df2['child_status'] = df2['children'].apply(child_status)
df2.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_def,days_employed_group,dob_years_group,income_group,child_status
21515,1,467,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486,заняться образованием,образование,стажер,рабочий возраст,синий воротничок,один-два ребенка
21516,0,914,42,высшее,0,женат / замужем,0,F,компаньон,0,322807,покупка своего жилья,недвижимость,постоянный работник,рабочий возраст,средний и высший класс,без детей
21517,0,404,42,высшее,0,гражданский брак,1,F,компаньон,0,178059,на покупку своего автомобиля,автомобиль,стажер,рабочий возраст,средний и высший класс,без детей
21518,0,15583,59,среднее,1,женат / замужем,0,F,пенсионер,0,153864,сделка с автомобилем,автомобиль,ударник,рабочий возраст,средний и высший класс,без детей
21519,1,2351,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949,покупка коммерческой недвижимости,коммерческая недвижимость,постоянный работник,рабочий возраст,синий воротничок,один-два ребенка
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость,постоянный работник,рабочий возраст,средний и высший класс,один-два ребенка
21521,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль,ударник,пенсионный возраст,средний и высший класс,без детей
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость,постоянный работник,рабочий возраст,синий воротничок,один-два ребенка
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль,постоянный работник,рабочий возраст,средний и высший класс,многодетные
21524,2,1984,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047,на покупку автомобиля,автомобиль,постоянный работник,рабочий возраст,синий воротничок,один-два ребенка


In [ ]:
child_dict = df2[['child_status']].drop_duplicates().reset_index(drop = True)
child_dict

# Results

# Number of children in correlation with on-time payments on loans.

In [ ]:
df2['debt'].loc[df2['debt'] == 1].sum()

1741

In [ ]:
child_to_debt = df2.pivot_table(index = 'child_status',values = 'debt', aggfunc = ['sum','count']).fillna(0)
child_to_debt.columns = ['debt','total']
child_to_debt['ratio,%'] = child_to_debt['debt'] / child_to_debt['total'] * 100
child_to_debt = child_to_debt.sort_values(by = ['ratio,%'], ascending = False)
child_to_debt


,debt,total,"ratio,%"
child_status,,,
один-два ребенка,647,6983,9.265359
многодетные,31,380,8.157895
без детей,1063,14090,7.544358




 <div class="alert alert-info">
   Better results in paying on-time we get from customers without kids, next would be large families with three and more children, but overall they not applying so much as other groups. And among parents of one or two kids typically 1 of 10 households gets to overdue payments.

   By comparasing we can say that risk of overdue loan for parents is 2% higher.     
    
    
    
    
 <div/>


# Correlation between martial status and on-time loan payment.

In [ ]:
family_to_debt = df2.pivot_table(index = family_dict,values = 'debt', aggfunc = ['sum','count']).fillna(0)
family_to_debt.columns = ['debt','total']
family_to_debt['ratio,%'] = family_to_debt['debt'] / family_to_debt['total'] * 100
family_to_debt = family_to_debt.sort_values(by = ['ratio,%'], ascending = False)
family_to_debt

,,debt,total,"ratio,%"
family_status,family_status_id,,,
Не женат / не замужем,4,274,2810,9.750890
гражданский брак,1,388,4150,9.349398
женат / замужем,0,931,12339,7.545182
в разводе,3,85,1195,7.112971
вдовец / вдова,2,63,959,6.569343




 <div class="alert alert-info">
  Loan applicants with unregistered relationtips such as civil marriage or single overdue payments usually 2% more often.


    
    
 <div/>


# Correlation between inclome and on-time payments.

In [ ]:
income_to_debt = df2.pivot_table(index = income_group_dict,values = 'debt', aggfunc = ['sum','count']).fillna(0)
income_to_debt.columns = ['debt','total']
income_to_debt['ratio,%'] = income_to_debt['debt'] / income_to_debt['total'] * 100
income_to_debt = income_to_debt.sort_values(by = ['ratio,%'], ascending = False)
income_to_debt

,debt,total,"ratio,%"
income_group,,,
синий воротничок,809,9368,8.635781
средний и высший класс,831,10611,7.831496
прожиточный минимум,101,1474,6.852103




 <div class="alert alert-info">
People with low inclome less tend to overdue payments than middle class salary employees.
High income loan applicants are more disciplined than middle class but still overdue is higher that low income applicants controvertially.

Seems like part of low income applicants might show only part of their inclome and therefore payments are not such a big pressure as it looks, whereas middle class tends to show all their income in paystub and what bank see is what they get actually, so they are more vulnerable to life circumstances.

    
    
 <div/>

# Correlation between loan purpose and on-time payments

In [ ]:
purpose_to_debt = df2.pivot_table(index = purpose_dict,values = 'debt', aggfunc = ['sum','count']).fillna(0)
purpose_to_debt.columns = ['debt','total']
purpose_to_debt['ratio,%'] = purpose_to_debt['debt'] / purpose_to_debt['total'] * 100
purpose_to_debt = purpose_to_debt.sort_values(by = ['ratio,%'], ascending = False)
purpose_to_debt

,debt,total,"ratio,%"
purpose_def,,,
автомобиль,403,4306,9.359034
образование,370,4013,9.220035
коммерческая недвижимость,99,1311,7.551487
потребительский,221,2930,7.542662
недвижимость,648,8893,7.286630




 <div class="alert alert-info">
Most of past-due payments are is car loans and student loans.
Best results we get in mortgage, consumer loans and commercial real estate.
Looks like car loans and student loans people tent to take in early stages of life where people tend to be not so discipline in payments and past-due occurs more often.

    
    
 <div/>

   <div class="alert alert-info">    <h3>Conclusions:</h3> based on data we got, we can presume, that more mature people with high-income, taking loans on mortgage and commercial real estate tend to pay better than rest.
    In common, average past-due risk for all categories is about 8%. </div>